# Confluence Dummy Template Scrape

## About

- Parses **'html file'** project page from confluence into a single .csv file.
- One must download the page as html through element inspector, save as html, then pass through this script.
- Tables are pulled from html as a list via pandas module.
- Tables are processed invidually using a range of custom functions
- Functions as expected as of 6th June 2018


## How to obtain 'html file'
- Load canvas page in confluence
- Right click, inspect element
- Right click on html body, copy inner code
- Paste in editor (notepad++, sublime etc)
- Save as .html
- Link path into this %%script

## Setup 

### Import Modules

In [1]:
import pandas
import bs4
import requests
import datetime

### Specify html file path and import as soup object, using html.parser

In [2]:
input_path = "/Users/danielcorcoran/Desktop/Confluence Files/inputs/project_dummy_page_number_3.html"

output_path = "/users/danielcorcoran/Desktop/Confluence Files/outputs/project_page/"

In [3]:
tables = pandas.read_html(input_path)

In [4]:
len(tables)

13

In [5]:
for index in range(len(tables)):
    print("Table at index:", index, "\n\n\n", tables[index], "\n\n\n","=" * 60)

Table at index: 0 


                    0                                                  1
0         Project ID                                                121
1      Project Title  Develop best practice data analytics lifecycle...
2        Last Update                                        06 Jun 2018
3        VCDI Stream                                        Data Reform
4       Project Lead                                   Natasha Thompson
5    Key Stakeholder                                         Brad Petry
6  Executive Sponsor                                      Julian Hebden
7      Project Start                                        01 Apr 2018
8        Project End                                        31 Dec 2018
9      Current State                                             Active 


Table at index: 1 


    Project Selection Score  Priority  Maturity  Complexity
0                      NaN       NaN       NaN         NaN 


Table at index: 2 


                   

## Functions
These functions will help mutate the tables into the desired look and feel, and perform repetitive tasks.


### Drop all rows and columns containing 100% nulls

In [6]:
def drop_null_rows_and_columns(dataframe):
    dataframe.dropna(how = "all", axis =1, inplace = True)
    dataframe.dropna(how = "all", axis =0, inplace = True)
    return dataframe

### Transpose table with two columns

In [7]:
def transpose_table_with_two_columns(table):

    table.columns = ["index", "values"]

    index_list = list(table["index"])
    table.index = index_list
    table.drop("index", axis = 1, inplace = True)

    table2 = table.transpose()
    table2.reset_index(inplace = True, drop = True)


    return table2

### Convert first row to headers and drop first row

In [8]:
def first_row_to_headers(table):
    
    table.columns = table.iloc[0].tolist()
    table.drop([0], axis = 0, inplace = True)
    
    return table

### Compress table vertically

In [9]:
def compress_table_vertically(table):
    
    table_dict = table.to_dict(orient = "list")

    new = {}
    for key in table_dict.keys():
        new[key] = ""
        for item in table_dict[key]:
            new[key] = new[key] + str(item) + " | "

    for key in new.keys():
        item = new[key]
        item_max_length = len(item)
        item2 = item[:item_max_length-3]
        new[key] = item2

    final_dictionary = {}

    for key in new.keys():
        final_dictionary[key] = [new[key]]

    data = pandas.DataFrame(final_dictionary)

    return data

### Reset table index and drop old index

In [10]:
def reset(table):
    table = table.reset_index(drop = True)
    return table

### Drop null column and rows in particular dataframes

In [11]:
relevant_table_indices = list(range(len(tables)))
relevant_table_indices.remove(1)
relevant_table_indices.remove(5)
relevant_table_indices.remove(2)
relevant_table_indices.remove(6)
for index in relevant_table_indices:
    tables[index] = drop_null_rows_and_columns(tables[index])

### Clean Project Code

In [12]:
def cleancode(var):
    var = str(var)
    var = var.strip()
    maxchar = len(var)
    if maxchar == 1:
        new_var = "00" + var
    elif maxchar == 2:
        new_var = "0" + var
    else:
        new_var = var
        
    return new_var

## Inspect and process each table

### Table 0 (KEY INFORMATION)

In [13]:
tables[0]

,0,1
0,Project ID,121
1,Project Title,Develop best practice data analytics lifecycle...
2,Last Update,06 Jun 2018
3,VCDI Stream,Data Reform
4,Project Lead,Natasha Thompson
5,Key Stakeholder,Brad Petry
6,Executive Sponsor,Julian Hebden
7,Project Start,01 Apr 2018
8,Project End,31 Dec 2018
9,Current State,Active


In [14]:
tables[0] = transpose_table_with_two_columns(tables[0])
tables[0]

,Project ID,Project Title,Last Update,VCDI Stream,Project Lead,Key Stakeholder,Executive Sponsor,Project Start,Project End,Current State
0,121,Develop best practice data analytics lifecycle...,06 Jun 2018,Data Reform,Natasha Thompson,Brad Petry,Julian Hebden,01 Apr 2018,31 Dec 2018,Active


In [15]:
project_code = cleancode(tables[0].iloc[0,0])
tables[0].iloc[0,0] = project_code

### Table 1 (PROJECT SCORES)

In [16]:
tables[1]

,Project Selection Score,Priority,Maturity,Complexity
0,NaN,NaN,NaN,NaN


### Table 2 (PROJECT LIFE CYCLE 1/2)

In [17]:
tables[2]

,0,1
0,1 - Pre Project,COMPLETE
1,2 - Feasibility,NaN
2,3 - Foundations,NaN
3,4 - Development,3-GREEN
4,5 - Delivery,NaN
5,6 - Closure,NaN
6,7 - Post Project,NaN


In [18]:
tables[2] = transpose_table_with_two_columns(tables[2])
tables[2]

,1 - Pre Project,2 - Feasibility,3 - Foundations,4 - Development,5 - Delivery,6 - Closure,7 - Post Project
0,COMPLETE,NaN,NaN,3-GREEN,NaN,NaN,NaN


### Table 3 (PROJECT LIFE CYCLE 2/2)

In [19]:
tables[3]

,0,1
0,Current Phase,Development
1,Overall Status,3-GREEN


In [20]:
tables[3] = transpose_table_with_two_columns(tables[3])
tables[3]

,Current Phase,Overall Status
0,Development,3-GREEN


###  Table 4 (STATUS UPDATE)

In [21]:
tables[4]

,0
0,Status Update
1,Exec summary and status of the project.


In [22]:
tables[4] = first_row_to_headers(tables[4])
tables[4]

,Status Update
1,Exec summary and status of the project.


In [23]:
tables[4] = reset(tables[4])
tables[4]

,Status Update
0,Exec summary and status of the project.


### Table 5 (DATA CATEGORIES)

In [24]:
tables[5]

,Data Type,Yes / No,Nature of Data / Info Used
0,Personal,No,NaN
1,Health,No,NaN


In [25]:
columns = list(tables[5].columns)
columns

['Data Type', 'Yes / No', 'Nature of Data / Info Used']

In [26]:
new_tbl5_dict = {}

In [27]:
for index in range(tables[5].shape[0]):
    datatype = tables[5].loc[index, columns[0]]
    type_header =  columns[1] + " " + datatype
    type_selection = tables[5].loc[index, columns[1]]
    info_header = columns[2] + " " + datatype
    info_selection = tables[5].loc[index, columns[2]]
    
    new_tbl5_dict[type_header] = [type_selection]
    new_tbl5_dict[info_header] = [info_selection]
new_tbl5_dict

{'Yes / No Personal': ['No'],
 'Nature of Data / Info Used Personal': [nan],
 'Yes / No Health': ['No'],
 'Nature of Data / Info Used Health': [nan]}

In [28]:
tbl5_data = pandas.DataFrame(new_tbl5_dict)
tables[5] = tbl5_data
tables[5]

,Yes / No Personal,Nature of Data / Info Used Personal,Yes / No Health,Nature of Data / Info Used Health
0,No,NaN,No,NaN


### Table 6 (STAKEHOLDERS)

In [29]:
tables[6]

,Stakeholder,Contacts / Description
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [30]:
tables[6]["Project ID"] = project_code

In [31]:
tables[6]

,Stakeholder,Contacts / Description,Project ID
0,NaN,NaN,121
1,NaN,NaN,121
2,NaN,NaN,121
3,NaN,NaN,121
4,NaN,NaN,121


In [32]:
null_stakeholder_column = tables[6]["Stakeholder"].isnull().sum()/tables[6].shape[0]

In [33]:
if null_stakeholder_column == 1:
    tables[6] = tables[6][:1]

### Table 7 (ARTIFACTS)

In [34]:
tables[7]

,0,1,2
0,#,Name / Wiki Link,CM Ref
1,01,Example,NaN
2,02,Example,F17/1234
4,Project Space(Click here to access project tea...,NaN,NaN


In [35]:
tables[7]

,0,1,2
0,#,Name / Wiki Link,CM Ref
1,01,Example,NaN
2,02,Example,F17/1234
4,Project Space(Click here to access project tea...,NaN,NaN


In [36]:
tables[7] = first_row_to_headers(tables[7])
tables[7]

,#,Name / Wiki Link,CM Ref
1,01,Example,NaN
2,02,Example,F17/1234
4,Project Space(Click here to access project tea...,NaN,NaN


In [37]:
tables[7] = compress_table_vertically(tables[7])
tables[7]

,#,Name / Wiki Link,CM Ref
0,01 | 02 | Project Space(Click here to access p...,Example | Example | nan,nan | F17/1234 | nan


### Table 8 (PROJECT OBJECTIVE OUTCOMES)

In [38]:
tables[8]

,0
0,Objective
1,The objective of the project.
2,Outcomes
3,Best practice data analytics lifecycle methodo...


In [39]:
dictionary = {tables[8].iloc[0,0] : [tables[8].iloc[1,0]],
             tables[8].iloc[2,0] : [tables[8].iloc[3,0]]}

dictionary

{'Objective': ['The objective of the project.'],
 'Outcomes': ['Best practice data analytics lifecycle methodology.']}

In [40]:
tables[8] = pandas.DataFrame(dictionary)
tables[8]

,Objective,Outcomes
0,The objective of the project.,Best practice data analytics lifecycle methodo...


### Table 9 (RISK REGISTER)

In [41]:
tables[9]

,Risk ID,Proj ID,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,RiskCategory,Title/Description,MitigationControlsTAPs,Impact,Likelihood,Rating,Owner,Status
0,1,0,18/05/18,18/05/18,Resourcing,Delivery ability of the analytics team in ligh...,abc,High,High,9,BP,Open
1,2,0,19/05/18,19/05/18,Financial,Limited budget,abcdef,High,High,9,BP,Closed


In [42]:
for index in range(tables[9].shape[0]):
    tables[9].iloc[index, 1] = project_code

In [43]:
tables[9] = compress_table_vertically(tables[9])
tables[9]

,Risk ID,Proj ID,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,RiskCategory,Title/Description,MitigationControlsTAPs,Impact,Likelihood,Rating,Owner,Status
0,1 | 2,121 | 121,18/05/18 | 19/05/18,18/05/18 | 19/05/18,Resourcing | Financial,Delivery ability of the analytics team in ligh...,abc | abcdef,High | High,High | High,9 | 9,BP | BP,Open | Closed


### Table 10 (ISSUES REGISTER)

In [44]:
tables[10]

,IssueID,ProjID,RiskRef,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,Issue Category,Title/Description,Resolution/Actions,Resolution due date,Impact,Priority,Owner,Status
0,1,0,2,19/05/18,19/05/18,Financial,Limited budget,NaN,21/05/18,Medium,High,BP,Open
1,2,0,1,20/05/18,20/05/18,Resourcing,Critical resource on leave,1.2.,22/05/18,Medium,High,BP,Closed


In [45]:
for index in range(tables[10].shape[0]):
    tables[10].iloc[index, 1] = project_code

In [46]:
tables[10] = compress_table_vertically(tables[10])
tables[10]

,IssueID,ProjID,RiskRef,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,Issue Category,Title/Description,Resolution/Actions,Resolution due date,Impact,Priority,Owner,Status
0,1 | 2,121 | 121,2 | 1,19/05/18 | 20/05/18,19/05/18 | 20/05/18,Financial | Resourcing,Limited budget | Critical resource on leave,nan | 1.2.,21/05/18 | 22/05/18,Medium | Medium,High | High,BP | BP,Open | Closed


### Table 11 (BENEFITS REGISTER)

In [47]:
tables[11]

,OBID,ProjID,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,OB Category,Title/Description,Realisation ETA,Priority,Owner,Achievement Status
0,1.0,0.0,20/05/18,20/05/18,Financial,Saved man hours,20/12/18,High,BP,Achieved


In [48]:
for index in range(tables[11].shape[0]):
    tables[11].iloc[index, 1] = project_code

In [49]:
tables[11] = compress_table_vertically(tables[11])
tables[11]

,OBID,ProjID,DateLoggeddd/mm/yy,DateUpdateddd/mm/yy,OB Category,Title/Description,Realisation ETA,Priority,Owner,Achievement Status
0,1.0,121,20/05/18,20/05/18,Financial,Saved man hours,20/12/18,High,BP,Achieved


### Table 12 (DATA REQUEST REGISTER)

In [50]:
tables[12]

,DRID,ProjID,DateRequesteddd/mm/yy,DateUpdateddd/mm/yy,DR Stakeholder,DR Stakeholder Contact,Data Type,Title/Description,Due Date,Date Received,Elapsed Time (from request date),Owner,Status
0,1.0,0.0,20/05/18,20/05/18,VBA,TBC,Place,Addresses,27/05/18,26/05/18,5 days,BP,Closed


In [51]:
for index in range(tables[12].shape[0]):
    tables[12].iloc[index, 1] = project_code

In [52]:
tables[12] = compress_table_vertically(tables[12])
tables[12]

,DRID,ProjID,DateRequesteddd/mm/yy,DateUpdateddd/mm/yy,DR Stakeholder,DR Stakeholder Contact,Data Type,Title/Description,Due Date,Date Received,Elapsed Time (from request date),Owner,Status
0,1.0,121,20/05/18,20/05/18,VBA,TBC,Place,Addresses,27/05/18,26/05/18,5 days,BP,Closed


## Combine All Tables

- Resulting table will be stakeholders table joined against combined remaining tables, minus the register and artifact tables.

### Combine tables 0,1,2,3,4,8 horizontally

In [53]:
combined_data1 = pandas.concat([tables[0],
                              tables[1], 
                              tables[2], 
                              tables[3], 
                              tables[4], 
                              tables[5],
                              tables[8]], axis = 1)

combined_data1

,Project ID,Project Title,Last Update,VCDI Stream,Project Lead,Key Stakeholder,Executive Sponsor,Project Start,Project End,Current State,...,7 - Post Project,Current Phase,Overall Status,Status Update,Yes / No Personal,Nature of Data / Info Used Personal,Yes / No Health,Nature of Data / Info Used Health,Objective,Outcomes
0,121,Develop best practice data analytics lifecycle...,06 Jun 2018,Data Reform,Natasha Thompson,Brad Petry,Julian Hebden,01 Apr 2018,31 Dec 2018,Active,...,NaN,Development,3-GREEN,Exec summary and status of the project.,No,NaN,No,NaN,The objective of the project.,Best practice data analytics lifecycle methodo...


In [54]:
combined_data2 = tables[6].merge(combined_data1,
                                 on = "Project ID",
                                 how = "left")

combined_data2

,Stakeholder,Contacts / Description,Project ID,Project Title,Last Update,VCDI Stream,Project Lead,Key Stakeholder,Executive Sponsor,Project Start,...,7 - Post Project,Current Phase,Overall Status,Status Update,Yes / No Personal,Nature of Data / Info Used Personal,Yes / No Health,Nature of Data / Info Used Health,Objective,Outcomes
0,NaN,NaN,121,Develop best practice data analytics lifecycle...,06 Jun 2018,Data Reform,Natasha Thompson,Brad Petry,Julian Hebden,01 Apr 2018,...,NaN,Development,3-GREEN,Exec summary and status of the project.,No,NaN,No,NaN,The objective of the project.,Best practice data analytics lifecycle methodo...


### Export 

In [55]:
filename = "combined_data_" + str(project_code) + ".csv"

combined_data2.to_csv(output_path + filename, index = False)